In [1]:
import torch
import Trainer
from network import NFM
import torch.utils.data as Data
from Utils.criteo_loader import getTestData, getTrainData

nfm_config = \
{
    'n_class':9,
    'linear_hidden1':2000,
    
    
    'dnn_hidden_units':[100,8],#MLP隐层
    'num_sparse_features_cols':10477,#the number of the gene columns
    'num_dense_features': 0,#dense features number
    'bi_dropout': 0.5,#Bi-Interaction 的dropout
    'num_epoch': 500,#训练epoch次数
    'batch_size': 16,#batch_size
    'lr': 1e-3,
    'l2_regularization': 1e-4,
    'device_id': 0,
    'use_cuda': False,
    'epoch':1000,
    
   
    'gene_name':'dataset/qiuguan/origin_800/gene_name.csv',
    'label_name':'dataset/qiuguan/origin_800/gene_label.csv'
    
}
#model definition
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(3300)
        self.fc1 = nn.Linear(3300, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.5)
    def forward(self, x):
        x=self.bn0(x)
        x = F.relu(self.drop(self.bn1(self.fc1(x))))
        x = F.relu(self.drop(self.bn2(self.fc2(x))))
        return F.softmax(self.bn3(self.fc3(x)), dim=1)
model = MLP().cuda()
print(model)

import os
import time
import argparse
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import torch.nn.functional as F  # 激励函数的库
#import network
import config
import evaluate
import data_utils
#import Trainer

def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output


import pandas as pd 

from sklearn.model_selection import train_test_split




def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output

class KZDatasetTest(data.Dataset):
    """ Construct the FM pytorch dataset. """
    #def __init__(self, file,label_file, feature_map,n_class=16):
    def __init__(self, csv_path):
    
       
        self.data_info = self.get_data_info(csv_path)
        
        
            
        
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
   
    
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',',header=None)
        
        df=df.iloc[1:,1:]
        #print("df:",df)
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        rows,cols=df.shape
        #print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(i))
        #print('labels:',labels)
        labels=np.array(labels)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)##############
            #print("i,data:",i,data)
            #data=pd.DataFrame(data,dtype=float)###############
            data=np.array(data)##
            
            label=labels[i]
            #print(data.shape)
            #print(label.shape)
            #label=label.tolist()
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
            
            
            data_info.append((data,label))
        return data_info
    
    
   
    
import torch
import torch.nn as nn
from torch.utils.data.dataset import *
from PIL import Image
from torch.nn import functional as F
import random
from sklearn.model_selection import train_test_split
import ast
import torchvision

MLP(
  (bn0): BatchNorm1d(3300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3300, out_features=2000, bias=True)
  (bn1): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=2000, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)


In [2]:
#######找特征基因#############从3301中找200个基因
#########################################################本次测试的目的是看200个基因的分类效果
##########测试步骤：从3301个基因中提取350个
############用200个构建新的分类模型
#################特征基因
######################为小球，根据上边的测试的基因个数，350最大
import torch

import torch.nn as nn
import torch.optim as optim


from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from mnist_test import Net, train, test


# Network parameters
class Params(object):
    batch_size = 64
    test_batch_size = 20
    epochs = 5
    lr = 0.01
    momentum = 0.5
    no_cuda = True
    seed = 1
    log_interval = 10
    
    def __init__(self):
        pass

args = Params()
torch.manual_seed(args.seed)
#device = torch.device("cpu")
device=torch.device('cuda')
kwargs = {}



##############数据准备
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
#功能：加载保存到path中的各层参数到神经网络
#path='dataset/qiuguan/model_new_K_fold_RandomTree/MLP_non_encode/mixed/MLP610.pkl'
#path='dataset/qiuguan/origin_800/non_encode_aug/aug_model/para_gause_0.4_0.2_0.6_0.2/MLP7910.pkl'##########use the final parameters as model,not the best parameters
path='dataset/qiuguan/origin_800/non_encode_aug/aug_with_simple/para_gause_0.8_0.6/MLP10710.pkl'#
#nfm=NFM(nfm_config)
mlp=MLP()
#print(nfm)
#net = nn.DataParallel(net)
#net = net.to(device)
mlp.load_state_dict(torch.load(path),strict=False)
mlp.cuda()

print(mlp)




mlp_params = list(mlp.named_parameters())
#print(nfm_params)
net=mlp
model=net###########
'''
testset = KZDatasetTest(csv_path='../NFM-pyorch-master/dataset/qiuguan/orign/')
   
test_loader = DataLoader(
         dataset=testset,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size']
        
     )
'''

testset_xiaoqiu  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoqiu/test_info.csv')#样本收集特征数据集，和测试数据集不同，这里边可能还包含训练集
   
test_loader_xiaoqiu = DataLoader(
         dataset=testset_xiaoqiu,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )

testset_xiaoguan  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoguan/test_info.csv')
   
test_loader_xiaoguan = DataLoader(
         dataset=testset_xiaoguan,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )

################小球
#find the topK genes_features for glomerulus from test samples which has good generality
#genes_features=[labels,the best prediction_score of glomerulus ,topK genes of glomerulus,the best prediction_score of kidney tubules ,topK genes of kidney tubules]
#note: this code should be runned(clicked) many times until every label has its topK genes,otherwise, the code of the next cell will error

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from innvestigator import InnvestigateModel
from utils import Flatten


inn_model = InnvestigateModel(model, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)

genes_features=np.array([i for i  in range(9)])
genes_features=genes_features.reshape(9,1).tolist()
model.double()
for data, target in test_loader_xiaoqiu:############小球

    data, target = data.to(device), target.to(device)
    #targets=torch.max(targets,1)[1]###################
    #print('data:',data.shape)
    batch_size = int(data.size()[0])
    #print('batch_size:',batch_size)#=20
    evidence_for_class = []
    #print("target:",target.shape)
    #print('target:',target[3])
    # Overlay with noise 
    # data[0] += 0.25 * data[0].max() * torch.Tensor(np.random.randn(28*28).reshape(1, 28, 28))
    #model_prediction, true_relevance = inn_model.innvestigate(in_tensor=data)

    for i in range(9):#10类
    # Unfortunately, we had some issue with freeing pytorch memory, therefore 
    # we need to reevaluate the model separately for every class.
        model_prediction, input_relevance_values = inn_model.innvestigate2(in_tensor=data, rel_for_class=i,target=target)
        evidence_for_class.append(input_relevance_values)
    #print('input_relevance_values:',input_relevance_values.shape)
    #print('evidence_for_class:',len(evidence_for_class))
    evidence_for_class = np.array([elt.numpy() for elt in evidence_for_class])
    #print('evidence_for_class:',evidence_for_class.shape)#[10,20,784]
    for idx, example in enumerate(data):#batch 中的每一个样本
        #print('example:',example.shape)
        prediction = np.argmax(model_prediction.cpu().detach(), axis=1)#
        #print('prediction[idx]:',prediction[idx])
        #print(evidence_for_class[prediction[idx]][idx])
        #fig, axes = plt.subplots(3, 5)
        '''
        fig.suptitle("Prediction of model: " + str(prediction[idx]) + "({0:.2f})".format(
            100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())))
        '''
        prediction_value=prediction[idx]
        p_x=model_prediction[idx][model_prediction[idx].argmax()].exp()
        p_sum=model_prediction[idx].exp().sum()
        prediction_score=100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())
        #print('prediction_value:',prediction_value)
        #print('prediction_score:',prediction_score)
        #print('分子:',p_x)
        #print('分母：',p_sum)
        #uu=pr
        #print("torch.argmax:",torch.argmax(target[idx]))
        if len(genes_features[prediction_value])==1:#有值，但还没有添加预测分数和特征值，只有标签#prediction_value代表第几种疾病
            if prediction_value!=torch.argmax(target[idx]).cpu().detach()  :
                print('不合格****************:',prediction_value)
            if prediction_value==torch.argmax(target[idx]).cpu().detach()  :#预测正确
                genes_features[prediction_value].append(prediction_score)
                print('合格：',prediction_value)
                relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                index=torch.topk(relevance_score_for_every_pixel,20,largest=True)#基因个数50#####150
                #print('pixel_sorted:',index)
                genes_features[prediction_value].append(index.indices)#添加前50基因特征
        
        else: 
            if genes_features[prediction_value][1]<prediction_score:#如果值比已有值大，说明预测更准确
                if prediction_value==torch.argmax(target[idx]).cpu().detach():
                    print('&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&:',prediction_value)
                    genes_features[prediction_value].pop(2)#先删除特征值
                    genes_features[prediction_value].pop(1)#先删除预测分数
                
                    genes_features[prediction_value].append(prediction_score)
        
                    relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                    #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                    relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                    index=torch.topk(relevance_score_for_every_pixel,20,largest=True)#基因个数50  200
                    #print('pixel_sorted:',index)
                    genes_features[prediction_value].append(index.indices)
                    #genes_features[prediction_value].append(index)
print('qiu_____genes_features.shape:',genes_features)

MLP(
  (bn0): BatchNorm1d(3300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3300, out_features=2000, bias=True)
  (bn1): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=2000, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=100, out_features=9, bias=True)
  (bn3): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
relevance
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:

torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')


&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&: tensor(3)
&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&: tensor(8)
&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&: tensor(7)
&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&: tensor(6)
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(2, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')


In [3]:
#extract topK genes of 肾小球glomerulus: genes_features[i][2]
#save it as a csv file

for i in range(9):
    
    qiu_genes_tensor=genes_features[i][2]
    if i==0:
        mm=qiu_genes_tensor
    else:
        mm=torch.vstack((mm,qiu_genes_tensor))
print(mm.shape)
qiu_genes_numpy=mm.detach().numpy()
qiu_genes_df=pd.DataFrame(qiu_genes_numpy)
qiu_genes_df.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/qiu_genes_df.csv')

torch.Size([9, 20])


In [7]:
#find the topK genes_features for kidney tubules from test samples which has good generality
#genes_features=[labels,the best prediction_score of glomerulus ,topK genes of glomerulus,the best prediction_score of kidney tubules ,topK genes of kidney tubules]
#note: this code should be runned(clicked) many times until every label has its topK genes,otherwise, the code of the next cell will error

###########如果不能为每个疾病找到特征基因，就反复点击这段代码，直到找到为止
testset_xiaoguan  = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoguan/test_info.csv')
   
test_loader_xiaoguan = DataLoader(
         dataset=testset_xiaoguan,
         #transform=torchvision.transforms.ToTensor(),
         
         batch_size=nfm_config['batch_size'],
         shuffle=True
        
     )
#小管特征        
for data, target in test_loader_xiaoguan:############小管

    data, target = data.to(device), target.to(device)
    #targets=torch.max(targets,1)[1]###################
    #print('data:',data.shape)
    batch_size = int(data.size()[0])
    #print('batch_size:',batch_size)#=20
    evidence_for_class = []
    #print("target:",target.shape)
    #print('target:',target[3])
    # Overlay with noise 
    # data[0] += 0.25 * data[0].max() * torch.Tensor(np.random.randn(28*28).reshape(1, 28, 28))
    #model_prediction, true_relevance = inn_model.innvestigate(in_tensor=data)

    for i in range(9):#10类
    # Unfortunately, we had some issue with freeing pytorch memory, therefore 
    # we need to reevaluate the model separately for every class.
        model_prediction, input_relevance_values = inn_model.innvestigate2(in_tensor=data, rel_for_class=i,target=target)
        evidence_for_class.append(input_relevance_values)
    #print('input_relevance_values:',input_relevance_values.shape)
    #print('evidence_for_class:',len(evidence_for_class))
    evidence_for_class = np.array([elt.numpy() for elt in evidence_for_class])
    #print('evidence_for_class:',evidence_for_class.shape)#[10,20,784]
    for idx, example in enumerate(data):#batch 中的每一个样本
        #print('example:',example.shape)
        prediction = np.argmax(model_prediction.cpu().detach(), axis=1)#
        #print('prediction[idx]:',prediction[idx])
        #print(evidence_for_class[prediction[idx]][idx])
        #fig, axes = plt.subplots(3, 5)
        '''
        fig.suptitle("Prediction of model: " + str(prediction[idx]) + "({0:.2f})".format(
            100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())))
        
        '''
        
        prediction_value=prediction[idx]
        p_x=model_prediction[idx][model_prediction[idx].argmax()].exp()
        p_sum=model_prediction[idx].exp().sum()
        prediction_score=100*float(model_prediction[idx][model_prediction[idx].argmax()].exp()/model_prediction[idx].exp().sum())
        #print('prediction_value:',prediction_value)
        #print('prediction_score:',prediction_score)
        #print('分子:',p_x)
        #print('分母：',p_sum)
        #uu=pr
        #print("torch.argmax:",torch.argmax(target[idx]))
        if len(genes_features[prediction_value])==3:#有小球的值，但还没有添加小管预测分数和特征值，只有标签
            if prediction_value!=torch.argmax(target[idx]).cpu().detach()  :
                print('不合格****************:',prediction_value)
            if prediction_value==torch.argmax(target[idx]).cpu().detach()  :#预测正确
                genes_features[prediction_value].append(prediction_score)
                print('合格：',prediction_value)
                relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                index=torch.topk(relevance_score_for_every_pixel,20,largest=True)#基因个数50
                #print('pixel_sorted:',index)
                genes_features[prediction_value].append(index.indices)#添加前50基因特征
        
        else: 
            if genes_features[prediction_value][3]<prediction_score:#如果值比已有值大，说明预测更准确
                if prediction_value==torch.argmax(target[idx]).cpu().detach():
                    print('&&&&&&&&&&&&&&&&&&&合   格&&&&&&&&&&&&&&:',prediction_value)
                    genes_features[prediction_value].pop(4)#先删除特征值
                    genes_features[prediction_value].pop(3)#先删除预测分数
                
                    genes_features[prediction_value].append(prediction_score)
        
                    relevance_score_for_every_pixel=evidence_for_class[prediction[idx]][idx]
                    #print('relevance_score_for_every_pixel.shape:',relevance_score_for_every_pixel.tolist())
                    relevance_score_for_every_pixel=torch.from_numpy(relevance_score_for_every_pixel)
                    index=torch.topk(relevance_score_for_every_pixel,20,largest=True)#基因个数50
                    #print('pixel_sorted:',index)
                    genes_features[prediction_value].append(index.indices)
    print('guan______________genes_features.shape:',genes_features)       
    
        
        
    break
        # fig.savefig("./mnist_example{0}.png".format(idx))

torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(0, device='cuda:0')
torch.argmax(predict): tensor(8, device='cuda:0')
torch.argmax(predict): tensor(7, device='cuda:0')
torch.argmax(predict): tensor(1, device='cuda:0')
torch.argmax(predict): tensor(6, device='cuda:0')
torch.argmax(predict): tensor(3, device='cuda:0')
torch.argmax(predict): tensor(4, device='cuda:0')
torch.argmax(predict): tensor(5, device='cuda:0')


In [8]:
#extract topK genes of 肾小管kidney tubules: genes_features[i][4]
#save it as a csv file

for i in range(9):
    
    guan_genes_tensor=genes_features[i][4]
    if i==0:
        mm=guan_genes_tensor
    else:
        mm=torch.vstack((mm,guan_genes_tensor))
print(mm.shape)
guan_genes_numpy=mm.detach().numpy()
guan_genes_df=pd.DataFrame(guan_genes_numpy)
guan_genes_df.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/guan_genes_df.csv')

torch.Size([9, 20])


In [9]:
#保存筛选的特征基因
#compute the union of kidney tubules and glomerulus without the repeat genes
import itertools

qiu_guan_all=[]

for i in range(9):
    #print('i:',i)
    qiu_features=genes_features[i][2]
    guan_features=genes_features[i][4]
    #print(qiu_features)
    qiu_features=qiu_features.detach().numpy()
    guan_features=guan_features.detach().numpy()
    qiu_guan_features = list(set(itertools.chain(*[qiu_features,guan_features])))
    #print(len(qiu_guan_features))
    #qiu_guan_features.append(i)
    qiu_guan_all.append(qiu_guan_features)
#qiu_guan_all.append('label')
#qiu_guan_all.append(10478)


qiu_guan_all_np=np.array(qiu_guan_all)
qiu_guan_all_df=pd.DataFrame(qiu_guan_all_np)
qiu_guan_all_df.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/qiu_guan_all_selected_features.csv')

/home/zhengfang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:

#求出所有基因的个数，基因名数
##########对所有疾病的特征基因求交集
import itertools
import numpy as np
import pandas as pd 
f_genes='dataset/qiuguan/origin_800/LRP/20/qiu_guan_all_selected_features.csv'
f_genes_df=pd.read_csv(f_genes,sep=',',header=None)
#print(f_genes_df)
f_genes_df=f_genes_df.iloc[1:,1:]
#print(f_genes_df)
#f_genes_df=f_genes_df.astype(int)
#f_genes_df=f_genes_df.values

qiu_guan_all=f_genes_df
qiu_guan_all=np.array(qiu_guan_all)
##qiu_guan_all.dtype='int'
#print(qiu_guan_all)
for i in range(9):
    #print('i:',i)
    qiu_guan_all_list=qiu_guan_all[i][0]
    #qiu_guan=qiu_guan[:-1]
    #print(qiu_guan)
    #=qiu_guan_all_list[0]
    qiu_guan_all_list_to_int = qiu_guan_all_list[1:-1]#去掉方括号，已经不是列表了，把方括号整体当作一个字符了
    
    
    
    
    qiu_guan_all_list_to_int=qiu_guan_all_list_to_int.split(',')#按，分割
    qiu_guan_all_list_to_int=[int(j) for j in qiu_guan_all_list_to_int]##########get the genes of a category
    #print(qiu_guan_all_list_to_int)
    #guan_features=genes_features[i][4]
    #print(qiu_features)
    #qiu_features=qiu_features.detach().numpy()
    #guan_features=guan_features.detach().numpy()
    #qiu_guan=[str(i) for i in qiu_guan]
    qiu_guan_disease=np.array(qiu_guan_all_list_to_int)
    qiu_guan_disease=pd.DataFrame(qiu_guan_disease)
    qiu_guan_disease.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/disease/qiu_guan_disease'+str(i)+'.csv')
    
    
    if i==0:
        qiu_guan_final=qiu_guan_all_list_to_int
    else:
        
        #qiu_guan_final=qiu_guan_final.
        #########get the genes of all categories
        qiu_guan_final = list(set(itertools.chain(*[qiu_guan_final,qiu_guan_all_list_to_int])))###########将球管合并，删除重复基因
#print(len(qiu_guan_final))
qiu_guan_final_len=len(qiu_guan_final)        

        #print(qiu_guan_features)
        #qiu_guan_features.append(i)
        #qiu_guan_all.append(qiu_guan_final)
#qiu_guan_all.append('label')
#qiu_guan_all.append(10478)
#print(qiu_guan_all)


df_all=pd.read_csv('dataset/qiuguan/origin_800/xiaoguan/test_info.csv',sep=',')
df_all=df_all.iloc[:,1:]
#print(df_all)
columns=df_all.columns
#print(columns)
#print(qiu_guan_final)
qiu_guan_final_columns=columns[qiu_guan_final]
print(qiu_guan_final_columns)


qiu_guan_final_columns=np.array(qiu_guan_final_columns).reshape(1,qiu_guan_final_len)
#print(qiu_guan_final.shape)
qiu_guan_final_columns=pd.DataFrame(qiu_guan_final_columns)
#print(qiu_guan_final)
qiu_guan_final_columns.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/qiu_guan_all_names_new_df_1.csv')

Index(['ABCC3', 'LRP5', 'LIPC', 'FKBP4', 'TFAM', 'ABHD2', 'CRABP2', 'NPIPB15',
       'ABLIM1', 'ACAD10',
       ...
       'NDUFA13', 'KLF6', 'AADAC', 'MST1L', 'SERPINF1', 'CAMLG', 'ABCB4',
       'CYP46A1', 'ABCB7', 'TYR'],
      dtype='object', length=134)


In [11]:
#######训练集 ：合起来：一部分
all_qiuguan=pd.read_csv('dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/train_val_info.csv',sep=',')

all_qiuguan=all_qiuguan.iloc[:,1:]
print('all:',all_qiuguan)


#X=X.values
#all_rows,all_cols=all_qiuguan.shape

###########测试集：分两部分：小球和小管

all_xiaoguan=pd.read_csv('dataset/qiuguan/origin_800/xiaoguan/test_info.csv',sep=',')

all_xiaoguan=all_xiaoguan.iloc[:,1:]
#print('all_xiaoguan:',all_xiaoguan)
#print('shape:',all_xiaoguan.shape)


###########测试集：分两部分：小球和小管

all_xiaoqiu=pd.read_csv('dataset/qiuguan/origin_800/xiaoqiu/test_info.csv',sep=',')

all_xiaoqiu=all_xiaoqiu.iloc[:,1:]
#print('all_xiaoqiu:',all_xiaoqiu)
#print('shape:',all_xiaoqiu.shape)



#all_rows_name=all.index=['a'+str(i) for i in range(all_rows)]
#print(rows_name)

#print(all)
column_name=pd.read_csv('dataset/qiuguan/origin_800/LRP_aug/20/qiu_guan_all_names_new_df_1.csv',sep=',')
#print('column_names:',column_name)
column_name=column_name.iloc[:,1:]
row_num,col_num=column_name.shape
column_name_list=np.array(column_name).reshape(col_num).tolist()
column_name_list.append('label')#############
print('column_names:',column_name_list)


all_qiuguan=all_qiuguan.loc[:,column_name_list]
#print(all_qiuguan)
all_xiaoguan=all_xiaoguan.loc[:,column_name_list]
all_xiaoqiu=all_xiaoqiu.loc[:,column_name_list]
print(all_xiaoqiu.head())
all_qiuguan.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/selected_train_val_info.csv')
all_xiaoguan.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/selected_xiaoguan_test_info.csv')
all_xiaoqiu.to_csv('dataset/qiuguan/origin_800/LRP_aug/20/selected_xiaoqiu_test_info.csv')

all:           RHOA      STX2      CISD1     WDR11     SCYL2   MGC2889     CCDC47  \
0    11.249338  6.118217   9.607851  7.381154  5.932984  4.568555   8.114415   
1    10.898457  5.905008  10.043073  7.461420  6.575100  4.241700   8.489481   
2    10.911331  6.069016  10.299607  7.182738  5.973380  4.809875   8.189254   
3    10.993798  5.980454   9.853507  7.477239  6.933372  4.576204   8.669412   
4    11.116181  6.223586  10.276357  7.721223  6.903274  4.518367   8.622838   
..         ...       ...        ...       ...       ...       ...        ...   
544  12.143129  8.564954   9.422492  8.952287  6.777686  5.370823  10.227443   
545  12.726252  8.631955   9.292412  8.955966  7.429243  4.985910  10.532490   
546  12.265202  8.325337   8.159097  8.508012  7.503678  5.299556   9.695292   
547  12.524550  8.936505   9.154908  9.581060  6.652090  5.280697  10.231094   
548  12.447256  9.023328   8.866550  9.026690  7.250243  5.410700  10.175947   

         KLF8      CCL1   SLCO3A1 